In [ ]:
using TracyWidomBeta,LinearAlgebra,Statistics,SparseArrays,Distributions,Trapz,ApproxFun,SpecialFunctions
using RandomMatrices,Plots,LaTeXStrings,TypedTables,StatsBase,FFTW

## Evolution of the density

Recall that in the beginning of spectral discretization, with $H(x,\theta)=\int_{0}^{\theta} \rho(x,\theta')\, d\theta'$, we have the following PDE for $\rho$:

$$\begin{align*}
\frac {\partial \rho}{\partial x}+\left(\frac {8}{\beta}\sin^{3}\theta \cos\theta\right)\frac {\partial \rho}{\partial \theta}+\left(\frac {2}{\beta}\sin^{4} \theta\right)\frac {\partial^{2} \rho}{\partial \theta^{2}}
+\left[\left(2x+2\right)\sin \theta \cos \theta+\frac {2}{\beta}\left(2\sin^{2}\theta \cos {2\theta}+2\sin \theta\cos \theta \sin {2\theta}\right)\right]\rho
+\left[\left(x+\frac {2}{\beta}\sin 2\theta\right)\sin^{2}\theta-\cos^{2}\theta\right]\frac {\partial \rho}{\partial \theta}=0.
\end{align*}$$

The following waterfall plots show the evolution of $\rho:=\frac {\partial H}{\partial \theta}$ as $x$ decreases from $x=2$ to $-10$ with step size $=-2$. These plots are generated by solving the above PDE using finite-difference discretization with $\Delta x=-10^{-3}$, $M=10^4$, and $\theta_{M}=10\pi$.

In [ ]:
function density_theta(β,l;x0=13.0,xN=-10.0,Δx=-0.001,M=10^4)
    
    # Set up the time domain
    t=x0:Δx:xN
    
    # Set up the initial condition
    Φ = x -> erf.(x/sqrt(2))/2 + 0.5
    dϕ = x -> exp.(-x^2/2)*1/sqrt(2pi)
    g = (x0,β,t) -> Φ( (x0 - cot(t).^2)./sqrt.(4/β * cot.(t)) )
    h0 = (x0,β,t) -> t < pi/2 ? g(x0,β,t) : 1.0
    chain = (x0,β,t) -> (x0 + 3*cot.(t).^2).*csc.(t).*sec.(t)./sqrt.(16/β * cot.(t))
    dg = (x0,β,t) -> t ≈ 0.0 ? 0.0 : dϕ((x0 - cot(t).^2)./sqrt(4/β * cot.(t))).*chain(x0,β,t)
    dh0 = (x0,β,t) -> t < pi/2 ? dg(x0,β,t) : 0.0
    mgrid=(n,L) -> L*(1:n)/n
    θ=mgrid(M,l*pi)
    h=(l/M)*pi
    c0=map(t -> dh0(x0,β,t),θ)
    sol=zeros(Float64,length(t),length(c0))
    sol[1,:]=c0'
    
    # Set up the matrices
    T=spdiagm(0=>fill(-2.0,M),1=>fill(1.0,M-1),-1=>fill(1.0,M-1))
    tt=(-2*(sin.(θ)).^4)/(β*(h^2))
    T=spdiagm(0=>vec(tt))*T
    um1=ones(Int64,M-2,1);um1=vcat(um1,4);ud=zeros(Int64,M-1,1);ud=vcat(ud,-3);um2=zeros(Int64,M-3,1);um2=vcat(um2,-1)
    U=spdiagm(0=>vec(ud),1=>fill(-1.0,M-1),-1=>vec(um1),-2=>vec(um2))
    
    # Stepping forward in time
    for k=1:length(t)-1
        u1=(1/(2*h))*(8*((sin.(θ)).^3).*(cos.(θ))/β+(t[k].+(2*sin.(2*θ))/β).*(sin.(θ)).^2-(cos.(θ)).^2)
        U1=spdiagm(0=>vec(u1))*U
        u2=(1/(2*h))*(8*((sin.(θ)).^3).*(cos.(θ))/β+(t[k+1].+(2*sin.(2*θ))/β).*(sin.(θ)).^2-(cos.(θ)).^2)
        U2=spdiagm(0=>vec(u2))*U
        i1=-(2*t[k]+1)*sin.(θ).*cos.(θ)-(2/β)*((2*(sin.(θ)).^2).*cos.(2*θ)+2*sin.(θ).*cos.(θ).*sin.(2*θ))
        I1=spdiagm(0=>vec(i1))
        i2=-(2*t[k+1]+1)*sin.(θ).*cos.(θ)-(2/β)*((2*(sin.(θ)).^2).*cos.(2*θ)+2*sin.(θ).*cos.(θ).*sin.(2*θ))
        I2=spdiagm(0=>vec(i2))
        rhs=(I+(Δx/2)*T+(Δx/2)*U1+(Δx/2)*I1)*c0
        lhs=I-(Δx/2)*T-(Δx/2)*U2-(Δx/2)*I2
        c0=lhs\rhs
        sol[k+1,:]=c0'
    end
    return sol,θ,t
end

In [ ]:
function water_den(β,l)
    result=density_theta(β,l)
    final_interest=result[1]
    d=result[2]
    t=result[3]
    x=d
    δ=0.8
    slope = 2
    dt = -2
    sc = 200.0
    up = 12
    C = 3
    delt=(x[end]-x[1])/((10^3)*pi)
    X1=x[1]:delt:x[end]
    ts = 2:dt:-10
    m = length(ts)
    X2 = ts.-ts[1]
    tick_shift = 0.01
    axis_shift = 0.97
    X2 = X2/X2[end]*(δ*(m-1)).+ tick_shift
    X3 = X2/slope
    dd = zeros(length(ts),length(x))
    for i=1:length(ts)
        posi=findall(x->x==ts[i],t)[1]
        r=final_interest[posi,:]
        dd[i,:]=r
    end
    x_range = [x[1],x[end]+ δ/slope*m]

    p=Plots.plot()
    for j = m:-1:1
        Plots.plot!((x .+ δ*(j-1)/slope)./1.1, sc*dd[j,:] .+ δ*(j-1) .+ C, grid = false, legend = false, lw = 5, color = :black, 
            fillrange = δ*(j-1), fillcolor = :white, seriesalpha = 2, alpha = 0.9,xtickfontsize=15,ytickfontsize=15,legendfontsize=10)
    end

    ## draws the x axis
    Plots.plot!(X1./1.1, 0*X1.+0.5.+2, color = :black, yticks = false, xticks = false, xaxis = (false,x_range), 
        yaxis = (false,[0,δ*(m+5) + up]),xtickfontsize=15,ytickfontsize=15,legendfontsize=10)

    ## draws the t axis
    Plots.plot!((X3 .+ x[end].+0.1)./1.1, X2.+axis_shift.+2, color = :black, yticks = false, xticks = false, xaxis = (false,x_range), 
        yaxis = (false,[0,δ*(m+5) + up]),xtickfontsize=15,ytickfontsize=15,legendfontsize=10)

    ## labelling for the t axis
    for j = 1:2:length(X3)
        Plots.annotate!((X3[j].+ x[end].+0.1+0.2)./1.1, X2[j]+1.1+2, Plots.text("-"),xtickfontsize=15,ytickfontsize=15,
            legendfontsize=10)
        Plots.annotate!((X3[j].+ x[end].+0.3+0.5)./1.1, X2[j]+1.1+2, Plots.text(ts[j], :left,12),xtickfontsize=15,ytickfontsize=15,
            legendfontsize=10)
    end

    ## labelling for the x axis
    del=(x[end]-x[1])/(2*pi)
    X1=x[1]:del:x[end]
    for j = 1:length(X1)
        Plots.annotate!(X1[j]/1.1, 0.5+2, Plots.text("|",7),xtickfontsize=15,ytickfontsize=15,legendfontsize=10)
        Plots.annotate!((X1[j].+0.1)./1.1, -.8+2, Plots.text(round(X1[j]),12,:below),xtickfontsize=15,ytickfontsize=15,legendfontsize=10)
    end
    return p
end

For $\beta=3$, we have

In [ ]:
water_den(3,10)

For $\beta=5$, we have

In [ ]:
water_den(5,10)

For $\beta=6$, we have

In [ ]:
water_den(6,10)

For $\beta=7$, we have

In [ ]:
water_den(7,10)

##  Contour plot of the domain as $x$ decreases

The following contour plots show the evolution of $H(x,\theta)$ as $x$ decreases from $x=13$ to $-10$ in the finite-difference discretization setting. $\Delta x=-10^{-3}$ and $M=10^4$ are used.

In [ ]:
function contour_H(β;x0=13.0,xN=-10.0,Δx=-0.001,M=10^4)
    
    # Set up the time domain
    t=x0:Δx:xN
    
    # Set up the initial condition
    Φ = x -> erf.(x/sqrt(2))/2 + 0.5
    dϕ = x -> exp.(-x^2/2)*1/sqrt(2pi)
    g = (x0,β,t) -> Φ( (x0 - cot(t).^2)./sqrt.(4/β * cot.(t)) )
    h0 = (x0,β,t) -> t < pi/2 ? g(x0,β,t) : 1.0
    mgrid=(n,L) -> L*(1:n)/n
    θ=mgrid(M,pi)
    h=(1/M)*pi
    c0=map(t -> h0(x0,β,t),θ)
    F=zeros(length(t),length(θ))
    F[end,:]=c0'
    final_interest=zeros(length(t),1); final_interest[1]=1;
    
    # Set up the matrices
    T=spdiagm(0=>fill(-2.0,M),1=>fill(1.0,M-1),-1=>fill(1.0,M-1))
    tt=(-2*(sin.(θ)).^4)/(β*(h^2))
    T=spdiagm(0=>vec(tt))*T
    um1=ones(Int64,M-2,1);um1=vcat(um1,4);ud=zeros(Int64,M-1,1);ud=vcat(ud,-3);um2=zeros(Int64,M-3,1);um2=vcat(um2,-1)
    U=spdiagm(0=>vec(ud),1=>fill(-1.0,M-1),-1=>vec(um1),-2=>vec(um2))
    
    # Stepping forward in time
    for k=1:length(t)-1
        u1=(1/(2*h))*((t[k].+(2*sin.(2*θ))/β).*(sin.(θ)).^2-(cos.(θ)).^2)
        U1=spdiagm(0=>vec(u1))*U
        u2=(1/(2*h))*((t[k+1].+(2*sin.(2*θ))/β).*(sin.(θ)).^2-(cos.(θ)).^2)
        U2=spdiagm(0=>vec(u2))*U
        rhs=(I+(Δx/2)*T+(Δx/2)*U1)*c0
        lhs=I-(Δx/2)*T-(Δx/2)*U2
        c0=lhs\rhs
        F[end-k,:]=c0'
        final_interest[k+1]=c0[end]
    end
    t=reverse(t);
    p = Plots.contour(θ, t, F, fill=true,xtickfontsize=15,ytickfontsize=15,legendfontsize=10)
    return p
end

For $\beta=3$, we have

In [ ]:
contour_H(3)

For $\beta=5$, we have

In [ ]:
contour_H(5)

For $\beta=6$, we have

In [ ]:
contour_H(6)

For $\beta=7$, we have

In [ ]:
contour_H(7)